In [1]:
import pandas as pd
import polars as pl

# Read Data

In [4]:
import pandas as pd

# Read the CSV file into a DataFrame
df = pd.read_csv('NEAR_USDT_futures_data_2020.csv')

# Drop unused columns
df.drop(columns=['coin','unused', 'closetime'], inplace=True)

# Display the first few rows of the DataFrame
df

,opentime,openprice,highprice,lowprice,closeprice,volume,quotevolume,trades,taker_buy_volume,taker_buy_quote
0,1602748800000,1.0625,1.1872,1.0625,1.1169,1449407,1.655530e+06,9968,766784,8.779915e+05
1,1602752400000,1.1169,1.1301,1.0778,1.1179,1616432,1.789316e+06,8743,873713,9.684362e+05
2,1602756000000,1.1176,1.1650,1.1136,1.1570,1270406,1.441663e+06,8310,721173,8.190750e+05
3,1602759600000,1.1572,1.1575,1.1210,1.1279,481575,5.503787e+05,4446,227307,2.600841e+05
4,1602763200000,1.1286,1.1812,1.1025,1.1536,1835160,2.107977e+06,9266,934671,1.074642e+06
...,...,...,...,...,...,...,...,...,...,...
36456,1733990400000,7.0790,7.1800,7.0780,7.1800,1252786,8.950482e+06,20675,658195,4.702462e+06
36457,1733994000000,7.1790,7.2200,7.1460,7.1490,1535789,1.103534e+07,23363,709728,5.101661e+06
36458,1733997600000,7.1490,7.1560,7.0210,7.0380,2065246,1.461731e+07,35347,875286,6.193253e+06
36459,1734001200000,7.0380,7.0510,6.9450,7.0070,1712135,1.197202e+07,32840,797708,5.579540e+06


In [5]:
df_m = df.copy()

# Calculate Volatility
df_m['volatility'] = (df_m['highprice'] - df_m['lowprice']) / df_m['lowprice']

# Initialize the new column with zeros or NaN
df_m['next_4h_avg_volatility'] = 0

# Use a for loop to calculate the sum of shifts for the next 10 periods
for i in range(1, 5):  # Loop through shifts from -1 to -10
    df_m['next_4h_avg_volatility'] += df_m['volatility'].shift(-i)

# Divide the summed values by 10 to get the average
df_m['next_4h_avg_volatility'] /= 4

df_m.dropna(inplace=True)

df_m

,opentime,openprice,highprice,lowprice,closeprice,volume,quotevolume,trades,taker_buy_volume,taker_buy_quote,volatility,next_4h_avg_volatility
0,1602748800000,1.0625,1.1872,1.0625,1.1169,1449407,1.655530e+06,9968,766784,8.779915e+05,0.117365,0.049656
1,1602752400000,1.1169,1.1301,1.0778,1.1179,1616432,1.789316e+06,8743,873713,9.684362e+05,0.048525,0.052932
2,1602756000000,1.1176,1.1650,1.1136,1.1570,1270406,1.441663e+06,8310,721173,8.190750e+05,0.046157,0.053795
3,1602759600000,1.1572,1.1575,1.1210,1.1279,481575,5.503787e+05,4446,227307,2.600841e+05,0.032560,0.050324
4,1602763200000,1.1286,1.1812,1.1025,1.1536,1835160,2.107977e+06,9266,934671,1.074642e+06,0.071383,0.047671
...,...,...,...,...,...,...,...,...,...,...,...,...
36452,1733976000000,7.0530,7.1250,7.0400,7.0810,1549828,1.097683e+07,29710,682310,4.834212e+06,0.012074,0.015098
36453,1733979600000,7.0820,7.2150,7.0710,7.1270,2039678,1.460077e+07,34191,1225453,8.771590e+06,0.020365,0.012596
36454,1733983200000,7.1270,7.1680,7.0640,7.1220,1633077,1.162963e+07,26984,677472,4.826473e+06,0.014723,0.013723
36455,1733986800000,7.1210,7.1440,7.0670,7.0800,792438,5.630026e+06,17852,347334,2.468128e+06,0.010896,0.014814


# Features Engineering

In [99]:
df_features = df_m.copy()

# Get Rolling 4h data (helped)
df_features['openprice_4h'] = df_features['openprice'].shift(3)
df_features['highprice_4h'] = df_features['highprice'].rolling(window=4).max()
df_features['lowprice_4h'] = df_features['lowprice'].rolling(window=4).min()
df_features['volume_4h'] = df_features['volume'].rolling(window=4).sum()
df_features['quotevolume_4h'] = df_features['quotevolume'].rolling(window=4).sum()
df_features['trades_4h'] = df_features['trades'].rolling(window=4).sum()
df_features['taker_buy_volume_4h'] = df_features['taker_buy_volume'].rolling(window=4).sum()
df_features['taker_buy_quote_4h'] = df_features['taker_buy_quote'].rolling(window=4).sum()
df_features['4h_volatility'] = (df_features['highprice_4h'] - df_features['lowprice_4h']) / df_features['lowprice_4h']
df_features['current_4h_avg_volatility'] = df_features['volatility'].rolling(window=4).mean()

# (kinda help)
df_features['ma_10_current_4h_avg_volatility'] = df_features['current_4h_avg_volatility'].rolling(window=10).mean()
df_features['ma_20_current_4h_avg_volatility'] = df_features['current_4h_avg_volatility'].rolling(window=20).mean()
df_features['ma_50_current_4h_avg_volatility'] = df_features['current_4h_avg_volatility'].rolling(window=50).mean()

# lagged (kinda help)
df_features['std_10_volume'] = df_features['volume'].rolling(window=10).std()
df_features['std_20_volume'] = df_features['volume'].rolling(window=20).std()
df_features['std_50_volume'] = df_features['volume'].rolling(window=50).std()

df_features['sum_10_volume'] = df_features['volume'].rolling(window=10).sum()
df_features['sum_20_volume'] = df_features['volume'].rolling(window=20).sum()
df_features['sum_50_volume'] = df_features['volume'].rolling(window=50).sum()

# lagged (kinda help)
df_features['lag_price_change_1h'] = df_features['closeprice'] - df_features['closeprice'].shift(1)
df_features['lag_price_change_2h'] = df_features['closeprice'] - df_features['closeprice'].shift(2)
df_features['lag_price_change_4h'] = df_features['closeprice'] - df_features['closeprice'].shift(4)
df_features['lag_price_change_8h'] = df_features['closeprice'] - df_features['closeprice'].shift(8)

df_features['lag_volume_change_1h'] = df_features['volume'] - df_features['volume'].shift(1)
df_features['lag_volume_change_2h'] = df_features['volume'] - df_features['volume'].shift(2)
df_features['lag_volume_change_4h'] = df_features['volume'] - df_features['volume'].shift(4)
df_features['lag_volume_change_8h'] = df_features['volume'] - df_features['volume'].shift(8)

# Rolling (help a lot)
df_features['ma_10_volatility'] = df_features['volatility'].rolling(window=10).mean()
df_features['ma_20_volatility'] = df_features['volatility'].rolling(window=20).mean()
df_features['ma_50_volatility'] = df_features['volatility'].rolling(window=50).mean()

# (kinda help)
df_features['lag_volatility_change_1h'] = df_features['volatility'] - df_features['volatility'].shift(1)
df_features['lag_volatility_change_2h'] = df_features['volatility'] - df_features['volatility'].shift(2)
df_features['lag_volatility_change_4h'] = df_features['volatility'] - df_features['volatility'].shift(4)
df_features['lag_volatility_change_8h'] = df_features['volatility'] - df_features['volatility'].shift(8)

# (kinda help)
df_features['min_10_volatility'] = df_features['volatility'].rolling(window=10).min()
df_features['min_20_volatility'] = df_features['volatility'].rolling(window=20).min()
df_features['min_50_volatility'] = df_features['volatility'].rolling(window=50).min()

# (kinda help)
df_features['max_10_open'] = df_features['openprice'].rolling(window=10).max()
df_features['min_10_open'] = df_features['openprice'].rolling(window=10).min()
df_features['max_20_open'] = df_features['openprice'].rolling(window=20).max()
df_features['min_20_open'] = df_features['openprice'].rolling(window=20).min()
df_features['max_50_open'] = df_features['openprice'].rolling(window=50).max()
df_features['min_50_open'] = df_features['openprice'].rolling(window=50).min()

# (kinda help)
df_features['max_10_close'] = df_features['closeprice'].rolling(window=10).max()
df_features['min_10_close'] = df_features['closeprice'].rolling(window=10).min()
df_features['max_20_close'] = df_features['closeprice'].rolling(window=20).max()
df_features['min_20_close'] = df_features['closeprice'].rolling(window=20).min()
df_features['max_50_close'] = df_features['closeprice'].rolling(window=50).max()
df_features['min_50_close'] = df_features['closeprice'].rolling(window=50).min()

df_features['day'] = pd.to_datetime(df_features['opentime'], unit='ms').dt.day
df_features['hour'] = pd.to_datetime(df_features['opentime'], unit='ms').dt.hour

# Shadow Length (kinda help)
df_features['high_open_dif'] = df_features['highprice'] - df_features['openprice']
df_features['high_close_dif'] = df_features['highprice'] - df_features['closeprice']
df_features['low_open_dif'] = df_features['openprice'] - df_features['lowprice']
df_features['low_close_dif'] = df_features['closeprice'] - df_features['lowprice']

# Price-to-Volume Ratios (kinda help)
df_features['price_volume_ratio'] = (df_features['closeprice'] - df_features['openprice']) / df_features['volume']
df_features['quoteprice_volume_ratio'] = (df_features['quotevolume']) / (df_features['highprice'] - df_features['lowprice'])

# Time specific
df_features['hour_of_day'] = pd.to_datetime(df_features['opentime'], unit='ms').dt.hour
df_features['is_peak_hour'] = df_features['hour_of_day'].apply(lambda x: 1 if x in [8, 12, 16] else 0)

# Volume Volatility Interaction
df_features['volatility_volume'] = df_features['volatility'] * df_features['volume']
df_features['volatility_quotevolume_ratio'] = df_features['volatility'] / df_features['quotevolume']

# Bollinger bands
df_features['bollinger_upper_20'] = df_features['closeprice'].rolling(window=20).mean() + (2 * df_features['closeprice'].rolling(window=20).std())
df_features['bollinger_lower_20'] = df_features['closeprice'].rolling(window=20).mean() - (2 * df_features['closeprice'].rolling(window=20).std())
df_features['bollinger_width_20'] = df_features['bollinger_upper_20'] - df_features['bollinger_lower_20']

# Hourly Seasonality Volatility Ratio
df_features['hourly_volatility_ratio'] = df_features['volatility'] / df_features.groupby('day')['volatility'].transform('mean')

# Session-Based Volatility
df_features['session'] = pd.cut(df_features['hour'], bins=[0, 8, 16, 24], labels=[1, 2, 3])
df_features['session_volatility'] = df_features.groupby('session')['volatility'].transform('mean')

# Momentum Change
df_features['momentum_change_10'] = df_features['closeprice'].diff(1).rolling(window=10).sum()
df_features['momentum_change_20'] = df_features['closeprice'].diff(1).rolling(window=20).sum()
df_features['momentum_change_50'] = df_features['closeprice'].diff(1).rolling(window=50).sum()

df_features.dropna(inplace=True)
df_features

C:\Users\user\AppData\Local\Temp\ipykernel_20024\1748808095.py:103: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  df_features['session_volatility'] = df_features.groupby('session')['volatility'].transform('mean')


,opentime,openprice,highprice,lowprice,closeprice,volume,quotevolume,trades,taker_buy_volume,taker_buy_quote,...,volatility_quotevolume_ratio,bollinger_upper_20,bollinger_lower_20,bollinger_width_20,hourly_volatility_ratio,session,session_volatility,momentum_change_10,momentum_change_20,momentum_change_50
52,1602936000000,0.7994,0.8073,0.7857,0.7935,783374,6.238367e+05,6806,388355,3.097979e+05,...,4.406828e-08,0.895006,0.737644,0.157362,1.359021,2,0.021566,0.0398,-0.0826,-0.3635
53,1602939600000,0.7935,0.8040,0.7744,0.7841,1139354,9.015986e+05,9553,612633,4.856943e+05,...,4.239485e-08,0.883007,0.739133,0.143875,1.889538,2,0.021566,-0.0597,-0.1051,-0.3438
54,1602943200000,0.7840,0.8036,0.7752,0.7978,1905833,1.506094e+06,13371,1010611,7.994994e+05,...,2.432498e-08,0.869304,0.743956,0.125348,1.811064,2,0.021566,-0.0327,-0.0888,-0.3558
55,1602946800000,0.7967,0.8130,0.7887,0.7900,984310,7.858549e+05,5701,529272,4.236381e+05,...,3.920596e-08,0.857510,0.747570,0.109940,1.523083,2,0.021566,-0.0217,-0.0818,-0.4292
56,1602950400000,0.7899,0.7977,0.7560,0.7609,1452344,1.125279e+06,10245,605373,4.696621e+05,...,4.901782e-08,0.856856,0.742274,0.114583,2.726738,2,0.021566,-0.0467,-0.0595,-0.4078
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36452,1733976000000,7.0530,7.1250,7.0400,7.0810,1549828,1.097683e+07,29710,682310,4.834212e+06,...,1.099941e-09,7.108749,6.514951,0.593799,0.522480,1,0.019362,0.2830,0.5360,0.8200
36453,1733979600000,7.0820,7.2150,7.0710,7.1270,2039678,1.460077e+07,34191,1225453,8.771590e+06,...,1.394781e-09,7.143164,6.537836,0.605327,0.881262,1,0.019362,0.2750,0.5730,0.8190
36454,1733983200000,7.1270,7.1680,7.0640,7.1220,1633077,1.162963e+07,26984,677472,4.826473e+06,...,1.265950e-09,7.162178,6.577122,0.585056,0.637098,1,0.019362,0.2590,0.5830,0.7250
36455,1733986800000,7.1210,7.1440,7.0670,7.0800,792438,5.630026e+06,17852,347334,2.468128e+06,...,1.935287e-09,7.166396,6.623604,0.542791,0.471497,1,0.019362,0.2030,0.5070,0.6430


# ML Baseline (Linear Regression)

## Evaluation Function

In [7]:
import numpy as np
import polars as pl
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# Adjust the evaluate_baseline function to accept only y_pred and y_test
def evaluate_baseline(y_pred, y_test):
    # Convert lists to Numpy arrays
    pred = np.array(y_pred)
    test = np.array(y_test)

    # Evaluation Metrics
    me = np.mean(pred - test)  # Mean Error
    mae = mean_absolute_error(test, pred)  # Mean Absolute Error
    mape = np.mean(np.abs((test - pred) / test)) * 100  # Mean Absolute Percentage Error
    mpe = np.mean((test - pred) / test) * 100  # Mean Percentage Error
    rmse = np.sqrt(mean_squared_error(test, pred))  # Root Mean Squared Error

    r = r2_score(test, pred)  # Coefficient of Determination (R-squared)
    min_max_error = np.abs((np.min(pred) - np.min(test)) + 
                            (np.max(pred) - np.max(test)))  # Min-Max Error

    df = pl.DataFrame({
        f'volatility_prediction': pred,
        f'volatility_actual': test
    })

    df = df.with_columns(
        abs(pl.col(f'volatility_prediction') - pl.col(f'volatility_actual')).alias(f'dif_volatility')
    )

    # overall median abs error
    median_abs_err =  df[f'dif_volatility'].median()

    # overall var abs error
    var_abs_err =  df[f'dif_volatility'].var()

    # Results
    results = {
        "Overall Median Absolute Error" : median_abs_err,
        "Overall Variance Absolute Error" : var_abs_err,
        "Mean Error": me,
        "Mean Absolute Error ": mae,
        "Mean Absolute Percentage Error": mape,
        "Mean Percentage Error": mpe,
        "Root Mean Squared Error": rmse,
        "R-squared": r, 
        "Min-Max Error": min_max_error
    }
    
    return results, df

## Sliding Window

In [57]:
import time
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression

# Separate df into features and target
features_df = df_features.drop(['opentime', 'next_4h_avg_volatility'], axis=1)
target_df = df_features['next_4h_avg_volatility']

# Convert to NumPy arrays
X = features_df.values
y = target_df.values

# Specify parameters for the sliding window approach
num_predictions = 1   # Number of rows to predict
gap = 1               # Gap (number of rows to skip after each window)
max_windows = [100]    # Maximum number of windows to process
set_limit = True      # Set this to False to process all windows

# Define list of window sizes
window_sizes = [20000]

# List to store results
all_results = []

# List to store resultsAggregate
for max_window in max_windows:
# Loop through each window size
    for window_size in window_sizes:
        print(f'Processing window size: {window_size}')

        # Calculate the number of windows based on dataset size
        num_windows = len(X) - window_size - num_predictions

        # Adjust the starting point if set_limit is True
        start_index = 0
        if set_limit:
            # Only process the latest max_window
            start_index = max(0, len(X) - window_size - num_predictions - max_window)
            num_windows = min(num_windows, max_window)

        # Initialize lists to store actual and predicted values for this window size
        y_vals = []
        y_preds = []

        # Loop through each sliding window with the gap applied
        for window_number in range(start_index, start_index + num_windows, gap):
            
            start = window_number
            end = start + window_size
            X_train = X[start:end]
            y_train = y[start:end]

            X_train_mean = np.mean(X_train, axis=0)
            X_train_std = np.std(X_train, axis=0)
            X_train_normalized = (X_train - X_train_mean) / X_train_std

            # Get the column index for 'cur_avg_vol_4h' from features_df
            close_index = features_df.columns.get_loc('volatility')

            # Normalize y_train using the mean and std of cur_avg_vol_4h
            close_mean = X_train[:, close_index].mean()
            close_std = X_train[:, close_index].std()
            y_train_normalized = (y_train - close_mean) / close_std

            # Prepare validation data for prediction
            X_val = X[end:end + num_predictions]
            y_val = y[end:end + num_predictions]

            # Normalize validation data using the statistics from the training set
            X_val_normalized = (X_val - X_train_mean) / X_train_std

            # Track the start time of the window processing
            start_time = time.time()

            # Initialize and fit the model
            model = LinearRegression()
            model.fit(X_train_normalized, y_train_normalized)

            # Predict on validation data
            y_pred_val = model.predict(X_val_normalized)
            # Denormalize y_val and y_pred_val using the mean and std of cur_avg_vol_4h
            y_pred_val_denorm = y_pred_val * close_std + close_mean

            # Track the end time of the window processing
            end_time = time.time()

            # Append actual and predicted values to the lists
            y_vals.append(y_val[0])  # Assuming a single prediction per window
            y_preds.append(y_pred_val_denorm[0])  # Assuming a single prediction per window

        # Now that all windows for this window_size are processed, evaluate the baseline
        eval_results, _ = evaluate_baseline(y_preds, y_vals)

        # Append the window size and evaluation metrics to the results list
        all_results.append({
            'eval_size' : max_window,
            'window_size': window_size,
            **eval_results
        })

# Convert the results to a DataFrame for further analysis
results_df = pd.DataFrame(all_results)

# Print the results DataFrame
results_df

Processing window size: 20000


,eval_size,window_size,Overall Median Absolute Error,Overall Variance Absolute Error,Mean Error,Mean Absolute Error,Mean Absolute Percentage Error,Mean Percentage Error,Root Mean Squared Error,R-squared,Min-Max Error
0,100,20000,0.004554,0.000146,-0.001504,0.007392,29.455144,-10.364196,0.014115,0.20872,0.043843


## Direct Random Split

In [60]:
import time
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split

# Separate df into features and target
features_df = df_features.drop(['opentime', 'next_4h_avg_volatility'], axis=1)
target_df = df_features['next_4h_avg_volatility']

# Convert to NumPy arrays
X = features_df.values
y = target_df.values

# Define list of train-test split proportions
train_test_ratios = [0.99]  # Example proportions for training set

# List to store results
all_results = []

# Loop through each train-test ratio
for train_ratio in train_test_ratios:
    print(f'Processing train-test split ratio: {train_ratio}')

    # Perform train-test split
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, train_size=train_ratio, random_state=42
    )

    # Normalize training data
    X_train_mean = np.mean(X_train, axis=0)
    X_train_std = np.std(X_train, axis=0)
    X_train_normalized = (X_train - X_train_mean) / X_train_std

    # Get the column index for 'volatility' from features_df
    close_index = features_df.columns.get_loc('volatility')

    # Normalize y_train using the mean and std of the 'volatility' column
    close_mean = X_train[:, close_index].mean()
    close_std = X_train[:, close_index].std()
    y_train_normalized = (y_train - close_mean) / close_std

    # Normalize test data using the training set's statistics
    X_test_normalized = (X_test - X_train_mean) / X_train_std

    # Initialize and fit the model
    start_time = time.time()
    model = LinearRegression()
    model.fit(X_train_normalized, y_train_normalized)

    # Predict on the test set
    y_pred_test = model.predict(X_test_normalized)
    # Denormalize predictions
    y_pred_test_denorm = y_pred_test * close_std + close_mean

    # Track the end time of the process
    end_time = time.time()

    # Evaluate the model
    eval_results, _ = evaluate_baseline(y_pred_test_denorm, y_test)

    # Append results to the list
    all_results.append({
        'train_ratio': train_ratio,
        **eval_results
    })

# Convert results to a DataFrame for further analysis
results_df = pd.DataFrame(all_results)

# Print the results DataFrame
results_df


Processing train-test split ratio: 0.99


,train_ratio,Overall Median Absolute Error,Overall Variance Absolute Error,Mean Error,Mean Absolute Error,Mean Absolute Percentage Error,Mean Percentage Error,Root Mean Squared Error,R-squared,Min-Max Error
0,0.99,0.00374,0.000025,0.000504,0.00513,28.712317,-13.422649,0.007129,0.577197,0.007057


## Chronological Split

### Filtering Data

In [54]:
import datetime

# Define start date
start_date = datetime.datetime(2024, 6, 1)  # Use datetime.datetime for timestamp()

# Convert to Unix timestamp
unix_start_date = int(start_date.timestamp())  # Converts to seconds since epoch

# Filter the DataFrame
df_features = df_features[df_features['opentime'] >= unix_start_date*1000]

df_features

,opentime,openprice,highprice,lowprice,closeprice,volume,quotevolume,trades,taker_buy_volume,taker_buy_quote,...,ma_20_volatility_volume,hourly_volatility_ratio,taker_maker_buy_volume_ratio,taker_maker_buy_quote_ratio,sum_10_taker_maker_buy_volume_ratio,sum_10_taker_maker_buy_quote_ratio,ma_10_taker_maker_buy_volume_ratio,ma_10_taker_maker_buy_quote_ratio,ma_20_taker_maker_buy_volume_ratio,ma_20_taker_maker_buy_quote_ratio
31792,1717200000000,7.251,7.279,7.243,7.267,433981,3.151872e+06,12560,224960,1633898.454,...,14246.999783,0.282436,1.076255,1.076369,0.924359,0.924577,0.924359,0.924577,0.931149,0.931720
31793,1717203600000,7.266,7.279,7.213,7.241,421731,3.053457e+06,11589,189155,1369571.081,...,14013.689919,0.517798,0.813304,0.813340,0.950144,0.950093,0.950144,0.950093,0.930784,0.931392
31794,1717207200000,7.242,7.267,7.232,7.247,363907,2.637378e+06,9683,181205,1313323.192,...,13801.702384,0.275044,0.991806,0.991895,0.949660,0.949277,0.949660,0.949277,0.927495,0.928068
31795,1717210800000,7.246,7.259,7.217,7.227,451866,3.269226e+06,9799,225167,1628930.172,...,13763.603991,0.330416,0.993242,0.993071,0.936107,0.935816,0.936107,0.935816,0.924674,0.925217
31796,1717214400000,7.226,7.235,7.207,7.223,333891,2.410899e+06,7895,160899,1162082.987,...,13586.032413,0.221008,0.930095,0.930548,0.934313,0.934067,0.934313,0.934067,0.929724,0.930324
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36426,1733882400000,6.167,6.436,6.145,6.407,1842808,1.159954e+07,43108,985495,6194625.123,...,117730.644394,2.117175,1.149516,1.146109,0.917692,0.917387,0.917692,0.917387,0.904357,0.903725
36427,1733886000000,6.406,6.501,6.392,6.439,1486724,9.601557e+06,30974,813831,5257917.985,...,116478.005077,0.785102,1.209451,1.210487,0.979464,0.978028,0.979464,0.978028,0.917044,0.916786
36428,1733889600000,6.440,6.471,6.377,6.433,750535,4.821506e+06,21899,392378,2521879.563,...,113165.059587,0.680199,1.095547,1.096647,0.961162,0.960403,0.961162,0.960403,0.926821,0.926949
36429,1733893200000,6.434,6.461,6.374,6.410,735627,4.713947e+06,18913,336143,2154629.652,...,111659.344662,0.630521,0.841443,0.841877,0.939955,0.939355,0.939955,0.939355,0.929343,0.929678


### Lin Reg Model

In [100]:
import time
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression

# Separate df into features and target
features_df = df_features.drop(['opentime', 'next_4h_avg_volatility'], axis=1)
target_df = df_features['next_4h_avg_volatility']

# Convert to NumPy arrays
X = features_df.values
y = target_df.values

# Define list of train-test split proportions
train_test_ratios = [0.93]  # Example proportions for training set

# List to store results
all_results = []

# Loop through each train-test ratio
for train_ratio in train_test_ratios:
    print(f'Processing train-test split ratio: {train_ratio}')

    # Split data chronologically
    split_index = int(len(X) * train_ratio)
    X_train, X_test = X[:split_index], X[split_index:]
    y_train, y_test = y[:split_index], y[split_index:]

    # Normalize training data
    X_train_mean = np.mean(X_train, axis=0)
    X_train_std = np.std(X_train, axis=0)
    X_train_normalized = (X_train - X_train_mean) / X_train_std

    # Get the column index for 'volatility' from features_df
    close_index = features_df.columns.get_loc('volatility')

    # Normalize y_train using the mean and std of the 'volatility' column
    close_mean = X_train[:, close_index].mean()
    close_std = X_train[:, close_index].std()
    y_train_normalized = (y_train - close_mean) / close_std

    # Normalize test data using the training set's statistics
    X_test_normalized = (X_test - X_train_mean) / X_train_std

    # Initialize and fit the model
    start_time = time.time()
    model = LinearRegression()
    model.fit(X_train_normalized, y_train_normalized)

    # Predict on the test set
    y_pred_test = model.predict(X_test_normalized)
    # Denormalize predictions
    y_pred_test_denorm = y_pred_test * close_std + close_mean

    # Track the end time of the process
    end_time = time.time()

    # Evaluate the model
    eval_results, _ = evaluate_baseline(y_pred_test_denorm, y_test)


    # Append results to the list
    all_results.append({
        'train_ratio': train_ratio,
        **eval_results,

    })

# Convert results to a DataFrame for further analysis
results_df = pd.DataFrame(all_results)

# Print the results DataFrame
results_df.T

Processing train-test split ratio: 0.93


,0
train_ratio,0.930000
Overall Median Absolute Error,0.003494
Overall Variance Absolute Error,0.000022
Mean Error,0.000591
Mean Absolute Error,0.004569
Mean Absolute Percentage Error,27.676354
Mean Percentage Error,-13.543327
Root Mean Squared Error,0.006564
R-squared,0.440039
Min-Max Error,0.047719


In [65]:
# Evaluate the model
eval_results, _ = evaluate_baseline(y_pred_test_denorm[-100:], y_test[-100:])

# Append results to the list
all_results.append({
    'train_ratio': train_ratio,
    **eval_results,

})

# Convert results to a DataFrame for further analysis
results_df = pd.DataFrame(all_results)

results_df.T

,0,1
train_ratio,0.930000,0.930000
Overall Median Absolute Error,0.003549,0.004280
Overall Variance Absolute Error,0.000023,0.000146
Mean Error,0.000512,-0.001009
Mean Absolute Error,0.004664,0.007700
Mean Absolute Percentage Error,28.268852,31.191298
Mean Percentage Error,-13.396960,-12.446193
Root Mean Squared Error,0.006677,0.014291
R-squared,0.424360,0.188804
Min-Max Error,0.047609,0.047014


In [66]:
# Define the condition
condition = y_test[-100:] >= 0.02

# Calculate the percentage of values meeting the condition
percentage = (condition.sum() / len(y_test[-100:])) * 100

# Print the result
print(f"Percentage of values >= 0.02: {percentage:.2f}%")

Percentage of values >= 0.02: 52.00%


### Turn value to class

In [101]:
import time
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression

# Separate df into features and target
features_df = df_features.drop(['opentime', 'next_4h_avg_volatility'], axis=1)
target_df = df_features['next_4h_avg_volatility']

# Convert to NumPy arrays
X = features_df.values
y = target_df.values

# Define list of train-test split proportions
train_test_ratios = [0.93]  # Example proportions for training set

# List to store results
all_results = []

# Loop through each train-test ratio
for train_ratio in train_test_ratios:
    print(f'Processing train-test split ratio: {train_ratio}')

    # Split data chronologically
    split_index = int(len(X) * train_ratio)
    X_train, X_test = X[:split_index], X[split_index:]
    y_train, y_test = y[:split_index], y[split_index:]

    # Normalize training data
    X_train_mean = np.mean(X_train, axis=0)
    X_train_std = np.std(X_train, axis=0)
    X_train_normalized = (X_train - X_train_mean) / X_train_std

    # Get the column index for 'volatility' from features_df
    close_index = features_df.columns.get_loc('volatility')

    # Normalize y_train using the mean and std of the 'volatility' column
    close_mean = X_train[:, close_index].mean()
    close_std = X_train[:, close_index].std()
    y_train_normalized = (y_train - close_mean) / close_std

    # Normalize test data using the training set's statistics
    X_test_normalized = (X_test - X_train_mean) / X_train_std

    # Initialize and fit the model
    start_time = time.time()
    model = LinearRegression()
    model.fit(X_train_normalized, y_train_normalized)

    # Predict on the test set
    y_pred_test = model.predict(X_test_normalized)
    # Denormalize predictions
    y_pred_test_denorm = y_pred_test * close_std + close_mean

    # # Apply the condition to y_test and y_pred_test_denorm
    y_test = np.where(y_test >= 0.02, 1, 0)
    y_pred_test_classified = np.where(y_pred_test_denorm >= 0.02, 1, 0)

    # Track the end time of the process
    end_time = time.time()


# Convert results to a DataFrame for further analysis
results_df = pd.DataFrame({
    'y_test': y_test,
    'y_pred_test_classified': y_pred_test_classified
})

# Print the results DataFrame
results_df

Processing train-test split ratio: 0.93


,y_test,y_pred_test_classified
0,1,0
1,1,0
2,1,0
3,1,1
4,1,1
...,...,...
2438,0,0
2439,0,1
2440,0,0
2441,0,0


In [102]:
import pandas as pd
from sklearn.metrics import accuracy_score, f1_score

# Calculate Accuracy
accuracy = accuracy_score(results_df['y_test'], results_df['y_pred_test_classified'])

# Calculate F1 Score
f1 = f1_score(results_df['y_test'], results_df['y_pred_test_classified'])

# Print results
print(f"Accuracy: {accuracy:.2f}")
print(f"F1 Score: {f1:.2f}")

Accuracy: 0.81
F1 Score: 0.69


In [103]:
from sklearn.metrics import accuracy_score, f1_score

# Take the last 100 rows
results_df_last_100 = results_df.tail(100)

# Calculate Accuracy
accuracy = accuracy_score(results_df_last_100['y_test'], results_df_last_100['y_pred_test_classified'])

# Calculate F1 Score
f1 = f1_score(results_df_last_100['y_test'], results_df_last_100['y_pred_test_classified'])

# Print results
print(f"Accuracy (Last 100): {accuracy:.2f}")
print(f"F1 Score (Last 100): {f1:.2f}")

Accuracy (Last 100): 0.73
F1 Score (Last 100): 0.73


In [9]:
# # Example: Take the last 100 rows
# y_pred_test_denorm_last_100 = y_pred_test_denorm[-100:]

# # Convert to a DataFrame
# df = pd.DataFrame(y_pred_test_denorm, columns=['prediction_value'])

# # Save to CSV
# output_file = 'pred_value_20241712.csv'
# df.to_csv(output_file, index=False)

# print(f"Saved the last 100 rows to '{output_file}'")

Saved the last 100 rows to 'pred_value_20241712.csv'
